In [1]:
import pandas as pd
import numpy as np

In [2]:
# import pandas as pd

# # Load the CSV file
# df = pd.read_csv('/kaggle/input/datamerge/merged_data_kaggle.csv')  # Update 'your_file.csv' with the actual file path

# # Initialize an empty DataFrame to store the balanced data
# balanced_df = pd.DataFrame()

# # Define the list of ethnicities as numerical values
# ethnicity_values = [1, 2, 3, 4]

# # Iterate over each ethnicity value
# for ethnicity_value in ethnicity_values:
#     # Filter the data for the current ethnicity value and select 40,000 random samples
#     ethnicity_df = df[df['ethnicity'] == ethnicity_value].sample(n=40000, random_state=42)
    
#     # Append the filtered data to the balanced DataFrame
#     balanced_df = pd.concat([balanced_df, ethnicity_df])

# # Shuffle the balanced DataFrame
# balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

# # Save the balanced DataFrame to a new CSV file
# balanced_df.to_csv('/kaggle/working/balanced40.csv', index=False)

# print("Balanced data saved to 'balanced_data.csv'.")


In [3]:
df=pd.read_csv('/kaggle/input/balamced40/balanced40.csv')
df22=df
df22

,id,ethnicity,image_path
0,n006425,4,/kaggle/input/vggface2/train/n006425/0049_04.jpg
1,n002116,1,/kaggle/input/vggface2/train/n002116/0221_02.jpg
2,n005084,2,/kaggle/input/vggface2/train/n005084/0191_02.jpg
3,n003153,2,/kaggle/input/vggface2/train/n003153/0196_01.jpg
4,n004216,3,/kaggle/input/vggface2/train/n004216/0005_01.jpg
...,...,...,...
159995,n007374,3,/kaggle/input/vggface2/train/n007374/0150_02.jpg
159996,n001352,3,/kaggle/input/vggface2/train/n001352/0474_01.jpg
159997,n003572,4,/kaggle/input/vggface2/train/n003572/0256_02.jpg
159998,n000731,4,/kaggle/input/vggface2/train/n000731/0038_01.jpg


In [4]:
df22.shape

(160000, 3)

In [5]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Flatten, Dense
from keras.applications.vgg16 import preprocess_input
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

2024-03-30 14:42:46.624155: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-30 14:42:46.624250: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-30 14:42:46.738301: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
batch_size=32
X,y=df22.drop('ethnicity',axis=1),df22['ethnicity']
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [7]:
# Path to the manually downloaded VGG16 weights file
weights_path = '/kaggle/input/vgg-16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'

# Load VGG16 model without top (fully connected) layers and set the weights
base_model = VGG16(weights=weights_path, include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

In [8]:
# Create a new model on top of VGG16
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
output_layer = Dense(len(np.unique(y_train)), activation='softmax')(x)  # Adjust the output layer based on the number of classes

In [9]:
model = Model(inputs=base_model.input, outputs=output_layer)

In [10]:
# Additional code for debugging
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"Number of unique classes: {len(np.unique(y_train))}")
model.summary()

# Convert 'ethnicity_encoded' to string
df22['ethnicity_encoded'] = df22['ethnicity'].astype(str)

X_train shape: (128000, 2)
X_test shape: (32000, 2)
Number of unique classes: 4


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,926,596 (68.38 MB)

 Trainable params: 3,211,908 (12.25 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [11]:
# ImageDataGenerator for data augmentation during training
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# ImageDataGenerator for validation data (only normalization, no augmentation)
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Flow training images in batches using train_datagen generator
train_generator = train_datagen.flow_from_dataframe(
    dataframe=df22.loc[X_train.index],
    x_col="image_path",
    y_col="ethnicity_encoded",
    target_size=(224, 224),  # Update target_size based on the VGG16 input size
    batch_size=batch_size,
    class_mode='categorical'
)

Found 128000 validated image filenames belonging to 4 classes.


In [12]:
# Flow validation images in batches using validation_datagen generator
validation_generator = validation_datagen.flow_from_dataframe(
    dataframe=df22.loc[X_test.index],
    x_col="image_path",
    y_col="ethnicity_encoded",
    target_size=(224, 224),  # Update target_size based on the VGG16 input size
    batch_size=batch_size,
    class_mode='categorical'
)

Found 32000 validated image filenames belonging to 4 classes.


In [13]:
# Compile the model with the Adam optimizer instance
adam_optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=adam_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Calculate steps_per_epoch and validation_steps based on the number of samples and batch size
num_train_samples = len(X_train)
num_test_samples = len(X_test)
steps_per_epoch = num_train_samples // batch_size
validation_steps = num_test_samples // batch_size

# Train the model using fit
history = model.fit(
    x=train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_steps
)


Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-03-30 14:56:33.124295: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 4.63498, expected 3.85968
2024-03-30 14:56:33.124352: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 6.55784, expected 5.78254
2024-03-30 14:56:33.124361: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4: 6.61282, expected 5.83752
2024-03-30 14:56:33.124370: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 6.34499, expected 5.56968
2024-03-30 14:56:33.124378: E external/local_xla/xla/service/gpu/buffer_comparator.cc:11

   2/4000 ━━━━━━━━━━━━━━━━━━━━ 4:48 72ms/step - accuracy: 0.3359 - loss: 24.5346   

I0000 00:00:1711810608.757931      69 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


4000/4000 ━━━━━━━━━━━━━━━━━━━━ 3605s 896ms/step - accuracy: 0.4089 - loss: 2.5039 - val_accuracy: 0.6247 - val_loss: 0.9397
Epoch 2/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 0s 8us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/10


/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


4000/4000 ━━━━━━━━━━━━━━━━━━━━ 2151s 537ms/step - accuracy: 0.5981 - loss: 0.9871 - val_accuracy: 0.6371 - val_loss: 0.9123
Epoch 4/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 0s 4us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 2029s 506ms/step - accuracy: 0.6128 - loss: 0.9565 - val_accuracy: 0.6426 - val_loss: 0.9062
Epoch 6/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 0s 5us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 1966s 491ms/step - accuracy: 0.6204 - loss: 0.9427 - val_accuracy: 0.6485 - val_loss: 0.8957
Epoch 8/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 0s 5us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 1965s 490ms/step - accuracy: 0.6212 - loss: 0.9407 - val_accuracy: 0.6490 - val_loss: 0.8878
Epoch 10/10
4000/4000 ━━━━━━━━━━━

In [14]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(validation_generator, steps=validation_steps)
print(f'Test Accuracy: {test_acc * 100:.2f}%')
print(f'Loss:{test_loss*100:.2f}%')

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 85s 85ms/step - accuracy: 0.6489 - loss: 0.8914
Test Accuracy: 64.90%
Loss:88.78%


In [15]:
# Make predictions on the validation set
predictions = model.predict(validation_generator, steps=validation_steps)
predictions

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 80s 80ms/step


array([[0.0158556 , 0.18003261, 0.07179419, 0.7323175 ],
       [0.00653385, 0.8693877 , 0.04772526, 0.07635323],
       [0.05687292, 0.48081285, 0.2965119 , 0.16580231],
       ...,
       [0.06152891, 0.4697636 , 0.1685647 , 0.30014277],
       [0.02399346, 0.17082335, 0.08740427, 0.7177789 ],
       [0.00468923, 0.7236225 , 0.04564332, 0.22604492]], dtype=float32)

In [16]:
# Save the trained model
model.save('/kaggle/working/my_model2.keras')
